# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from z3 import *
from itertools import combinations
import sys
sys.path.append('../../scripts/biota/')
from BIoTA_Control import *
sys.path.append('../../scripts/shatter/')
from SHATTER_Control import *
import warnings
warnings.filterwarnings("ignore")
from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull, convex_hull_plot_2d

# Fixed Parameter

In [2]:
CO2_FRESH_AIR = 400             # CO2 concentration (ppm) of fresh air
TEMP_FRESH_AIR = 91.4           # Temperature (33 degree F) of the fresh air
CP_AIR = 1.026                  # Specific heat of fresh air
DEF_TEMP_SUPPLY_AIR =  55.4     # Default temperature (degree fahrenheit) of supply air (13 degree celsius)
COST_PER_KWH = 0.1137           # USD Cost per KWh

# Variable Parameters

In [3]:
zone_temp_setpoint = [0, 75.2, 75.2, 75.2, 75.2]     # list of temperature (fahrenheit) setpoint of the different zones
zone_co2_setpoint = [0, 1000, 1000, 1000, 1000]      # list of CO2 (ppm) setpoint of the corresponding zones
control_time = 1                                     # control time (in minute)

# Accessing Zone-Activity Information

In [4]:
zones = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Zone-Info')
zone_volume = zones["Volume (cf)"].to_list()                  # Zones' volumes (cubic feet)
pp_co2 = zones["CO2 Emission by Occupant (cfm)"].to_list()    # CO2 Emission by Occupant (cfm)
pp_heat = zones["Heat Radiation by Occupant (W)"].to_list()   # Heat Radiation by Occupant (W)
load = zones["Heat Radiated by Appliances (W)"].to_list()     # Heat radiated by Appliances (W)
        
activities = pd.read_excel('Aras-Information.xlsx', sheet_name='Activity-Info')

activity_zone_map = dict()
for i in range(len(activities)):
    activity_zone_map[int(activities["Activity ID"][i])] = int(activities["Zone ID"][i])

# Generating Possible Combination of Occupants

In [5]:
import itertools
all_samples = list(itertools.permutations([1, 1, 0, 0, 0]))
all_samples += list(itertools.permutations([2, 0, 0, 0, 0]))
# however, there are repetations

unique_samples = set()

for data in all_samples:    
    unique_samples.add(data)
    
unique_samples

{(0, 0, 0, 0, 2),
 (0, 0, 0, 1, 1),
 (0, 0, 0, 2, 0),
 (0, 0, 1, 0, 1),
 (0, 0, 1, 1, 0),
 (0, 0, 2, 0, 0),
 (0, 1, 0, 0, 1),
 (0, 1, 0, 1, 0),
 (0, 1, 1, 0, 0),
 (0, 2, 0, 0, 0),
 (1, 0, 0, 0, 1),
 (1, 0, 0, 1, 0),
 (1, 0, 1, 0, 0),
 (1, 1, 0, 0, 0),
 (2, 0, 0, 0, 0)}

# Tabulate the Control Costs for all Possible Combinations

In [6]:
# convert list to a tuple
def to_tuple(a):
    try:
        return tuple(to_tuple(i) for i in a)
    except TypeError:
        return a
    
dict_control_cost = dict()
for sample in unique_samples:
    zone_occupant = list(sample)    
    dict_control_cost[sample] = control_cost(zones, zone_occupant, zone_temp_setpoint, zone_volume, pp_co2, pp_heat, load, zone_co2_setpoint, control_time)
dict_control_cost

{(1, 0, 1, 0, 0): 0.0007955147465,
 (1, 1, 0, 0, 0): 0.0007418396295,
 (0, 0, 1, 0, 1): 0.00420112231555,
 (2, 0, 0, 0, 0): 0.0,
 (0, 0, 0, 2, 0): 0.0127561109392,
 (0, 0, 0, 0, 2): 0.0068112151381,
 (0, 1, 0, 0, 1): 0.00414744719855,
 (1, 0, 0, 0, 1): 0.00340560756905,
 (0, 0, 1, 1, 0): 0.0071735702161000005,
 (0, 2, 0, 0, 0): 0.001483679259,
 (0, 0, 0, 1, 1): 0.00978366303865,
 (0, 1, 0, 1, 0): 0.0071198950991,
 (0, 0, 2, 0, 0): 0.001591029493,
 (0, 1, 1, 0, 0): 0.0015373543760000001,
 (1, 0, 0, 1, 0): 0.0063780554696}

# Reading SHATTER Attack-Schedule Datasets

In [7]:
house_A_occ_1_final_schedule = pd.read_csv("../../data/shatter/House-A_Occ-1_SHATTER_Schedule.csv").values
house_A_occ_2_final_schedule = pd.read_csv("../../data/shatter/House-A_Occ-2_SHATTER_Schedule.csv").values
house_B_occ_1_final_schedule = pd.read_csv("../../data/shatter/House-B_Occ-1_SHATTER_Schedule.csv").values
house_B_occ_2_final_schedule = pd.read_csv("../../data/shatter/House-B_Occ-2_SHATTER_Schedule.csv").values
house_A_occ_1_greedy_schedule =  pd.read_csv("../../data/shatter/House-A_Occ-1_Greedy_Schedule.csv").values
house_A_occ_2_greedy_schedule =  pd.read_csv("../../data/shatter/House-A_Occ-2_Greedy_Schedule.csv").values
house_B_occ_1_greedy_schedule =  pd.read_csv("../../data/shatter/House-B_Occ-1_Greedy_Schedule.csv").values
house_B_occ_2_greedy_schedule =  pd.read_csv("../../data/shatter/House-B_Occ-2_Greedy_Schedule.csv").values

# SHATTER Attack Cost Calculation for House-A (Daily)

In [8]:
shatter_cost_house_A = []

for i in range(len(house_A_occ_1_final_schedule)):
    zone_occupants = [0, 0, 0, 0, 0]
    zone_occupants[int(house_A_occ_1_final_schedule[i])] += 1
    zone_occupants[int(house_A_occ_2_final_schedule[i])] += 1
    shatter_cost_house_A.append(dict_control_cost[to_tuple(zone_occupants)])  
sum(shatter_cost_house_A)

7.73624516809576

# SHATTER Control Cost Calculation for House-B (Daily)

In [9]:
shatter_cost_house_B = []

for i in range(len(house_B_occ_1_final_schedule)):
    zone_occupants = [0, 0, 0, 0, 0]
    zone_occupants[int(house_B_occ_1_final_schedule[i])] += 1
    zone_occupants[int(house_B_occ_2_final_schedule[i])] += 1
    shatter_cost_house_B.append(dict_control_cost[to_tuple(zone_occupants)])           
sum(shatter_cost_house_B)

6.534456791001837

# Greedy Attack Cost Calculation for House-A (Daily)

In [10]:
greedy_cost_house_A = []

for i in range(len(house_A_occ_1_greedy_schedule)):
    zone_occupants = [0, 0, 0, 0, 0]
    zone_occupants[int(house_A_occ_1_greedy_schedule[i])] += 1
    zone_occupants[int(house_A_occ_2_greedy_schedule[i])] += 1
    greedy_cost_house_A.append(dict_control_cost[to_tuple(zone_occupants)])  
sum(greedy_cost_house_A)

6.713968683633121

# Greedy Attack Cost Calculation for House-B (Daily)

In [11]:
greedy_cost_house_B = []

for i in range(len(house_B_occ_1_greedy_schedule)):
    zone_occupants = [0, 0, 0, 0, 0]
    zone_occupants[int(house_B_occ_1_greedy_schedule[i])] += 1
    zone_occupants[int(house_B_occ_2_greedy_schedule[i])] += 1
    greedy_cost_house_B.append(dict_control_cost[to_tuple(zone_occupants)])  
sum(greedy_cost_house_B) 

6.5126398592216574

# Accessing Zone-Activity-Appliances Information

In [12]:
zones = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Zone-Info')
zone_volume = zones["Volume (cf)"].to_list()                    
        
activities = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Activity-Info')
energy_appliances = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Appliances-Info').iloc[:,-1].to_numpy()

num_activities = len(activities)  

activity_zone_map = dict()
for i in range(num_activities):
    activity_zone_map[int(activities["Activity ID"][i])] = int(activities["Zone ID"][i])

benign_activity_appliances_map = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Benign-Activity-Appliances-Map').iloc[:, 4:].to_numpy()
  
activity_zone_temp_setpoint = [0]
activity_zone_co2_setpoint = [0] 
activity_pp_co2 = [0] + activities['CO2 Emission by Occupant (CFM)'].to_list()    # CO2 Emission by Occupant (cfm)
activity_pp_heat = [0] + activities["Heat Radiation by Occupant (W)"].to_list()   # Heat Radiation by Occupant (W)
activity_load = [0]# Heat radiated by Appliances (W)
activity_zone_volume = [0]

for i in range(1, num_activities + 1):
    activity_zone_temp_setpoint.append(zone_temp_setpoint[activity_zone_map[i]])
    activity_zone_co2_setpoint.append(zone_co2_setpoint[activity_zone_map[i]])
    activity_zone_volume.append(zone_volume[activity_zone_map[i]])
    activity_load.append(sum(benign_activity_appliances_map[i] * energy_appliances))

# Generating Possible Combination of Occupants

In [13]:
activities = [i for i in range(1, num_activities + 1)]

indexes = list(combinations(activities, 2))

unique_samples = np.zeros((len(indexes) + num_activities, num_activities + 1))

for i in range(len(indexes)):
    unique_samples[i][indexes[i][0]] = 1
    unique_samples[i][indexes[i][1]] = 1

count = 1
for i in range(len(indexes), len(indexes) + num_activities):
    unique_samples[i][count] = 2
    count += 1

# Tabulate the Control Costs for all Possible Combinations

In [14]:
# convert list to a tuple
def to_tuple(a):
    try:
        return tuple(to_tuple(i) for i in a)
    except TypeError:
        return a
    
dict_control_cost = dict()
for i in range(len(unique_samples)):
    print(i)
    sample = unique_samples[i]
    activity_zone_occupant = sample.tolist()
    dict_control_cost[to_tuple(sample)] = shatter_control_cost(activity_zone_occupant, activity_zone_temp_setpoint, activity_zone_volume, activity_pp_co2, activity_pp_heat, activity_load, activity_zone_co2_setpoint, control_time)
dict_control_cost

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

{(0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0): 0.0,
 (0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0): 0.010410543052605165,
 (0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0): 0.0008332246760292,
 (0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0): 0.010410543052605165,
 (0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0

# Zone-wise Appliances

In [15]:
zone_appliance = []
zone_appliance.append([1, 2]) #0
zone_appliance.append([11, 16, 17, 23, 27]) #1
zone_appliance.append([12, 13, 18, 22, 25, 26]) #2
zone_appliance.append([3, 4, 5, 6, 7, 8, 9, 10, 19, 24]) #3
zone_appliance.append([14, 15, 20, 21]) #4

for i in range(len(zone_appliance)):
    print(len(zone_appliance[i]))

2
5
6
10
4


# Zone-wise Maximum Cost For Appliances

In [16]:
zone_cost = [0, 0, 0, 0, 0]
for i in range(len(zone_cost)):
    all_costs = []
    for j in range(len(zone_appliance[i])):
        all_costs.append(activity_load[zone_appliance[i][j]])
    zone_cost[i] =  max(all_costs)
zone_cost

[0, 256, 270, 5127, 1518]

# Clustering and Convex Hull

In [18]:
# returns dbscan clusters
def dbscan(X, eps, min_samples):
    db = DBSCAN(eps=eps, min_samples=min_samples)
    cluster = db.fit(X)
    #y_pred = db.fit_predict(X)
    #plt.scatter(X[:,0], X[:,1],c=y_pred, cmap='Paired')
    #plt.title("DBSCAN")
    #print("noise", sum(cluster.labels_ ==-1), "regular", sum(cluster.labels_ !=-1))
    #print(np.unique(cluster.labels_))
    return cluster

# returns convex hull of points associated with zones
def convex_hull(zone, points):
        '''
        Parameters
        ----------
        points : Array
            A set of points.
            
        Returns
        -------
        vertices : Points
            Arranges convex hull vertices in counter-clockwise orientation.
        '''    
        
        hull = ConvexHull(points)
        
        
        #plt.plot(points[:,0], points[:,1], 'o')
        #plt.ylim(0,120)
        #for simplex in hull.simplices:
        #    plt.plot(points[simplex, 0], points[simplex, 1], 'k-')
            
        simplices = hull.simplices
        
        vertices = []
        for index in hull.vertices:
            vertices.append((points[index][0], zone, points[index][1]))
        vertices.append((points[hull.vertices[0]][0], zone, points[hull.vertices[0]][1]))
        
        return vertices

# Cluster Boundary Acquisition

In [19]:
def get_cluster(dataframe, column, eps, min_samples):
    list_cluster = []
    for zone in range(num_zones):
        #plt.figure()
        mod_dataframe = pd.DataFrame()

        for i in range(len(dataframe)):
            if int(dataframe[column][i] == zone):
                mod_dataframe = mod_dataframe.append(dataframe.loc[i, ['Start Time', 'Duration']])

        features = np.empty([len(mod_dataframe), 2])
        features[:, 0:1] = mod_dataframe.loc[:, ['Start Time']].values
        features[:, 1:] = mod_dataframe.loc[:, ['Duration']].values
        cluster_model = dbscan(features, eps, min_samples)
        labels = cluster_model.labels_
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        for cluster in range(n_clusters):
            points = []
            for k in range(len(labels)):
                if labels[k] == cluster:
                    points.append(features[k])
            points = np.array(points)

            if len(points) >= 3:
                vertices = convex_hull(zone, points)
            else:
                print("zone", zone, "Problem Found!")

            list_cluster.append({"zone_id": zone, "cluster_id": cluster, "points": vertices})

    return list_cluster

# Reading Clean Dataframes

In [20]:
cleaned_duration_dataframe_house_A_occ_1 = pd.read_csv("../../data/cleaned/Cleaned-Duration-Dataframe_House-A_Occ-1.csv")
cleaned_duration_dataframe_house_A_occ_2 = pd.read_csv("../../data/cleaned/Cleaned-Duration-Dataframe_House-A_Occ-2.csv")
cleaned_duration_dataframe_house_B_occ_1 = pd.read_csv("../../data/cleaned/Cleaned-Duration-Dataframe_House-B_Occ-1.csv")
cleaned_duration_dataframe_house_B_occ_2 = pd.read_csv("../../data/cleaned/Cleaned-Duration-Dataframe_House-B_Occ-2.csv")

# Get Cluster Information

In [22]:
num_zones = 5
clusters_house_A_occ_1 = get_cluster(cleaned_duration_dataframe_house_A_occ_1, "Occupant-1 Zone ID", 20, 3)
clusters_house_A_occ_2 = get_cluster(cleaned_duration_dataframe_house_A_occ_2, "Occupant-2 Zone ID", 20, 3)
clusters_house_B_occ_1 = get_cluster(cleaned_duration_dataframe_house_B_occ_1, "Occupant-1 Zone ID", 30, 3)
clusters_house_B_occ_2 = get_cluster(cleaned_duration_dataframe_house_B_occ_2, "Occupant-2 Zone ID", 30, 3)

# Range Claculation Function

In [23]:
def is_left( x, y, init_x, init_y, final_x, final_y):
     return ((final_x - init_x)*(y - init_y) - (final_y - init_y)*(x - init_x)) >= 0

def range_calculation(list_cluster):
    num_timeslots = 1440
    list_time_min = [[[] for j in range(num_timeslots)] for i in range(num_zones + 1)]
    list_time_max = [[[] for j in range(num_timeslots)] for i in range(num_zones + 1)]

    for i in range(len(list_cluster)):
        zone_id = list_cluster[i]["zone_id"]
        min_x_range = 1440
        max_x_range = 0

        ##################################################################
        ##################### Zone Constraints ###########################
        ##################################################################
        x = Int('x')
        y = Int('y')

        points = list_cluster[i]["points"]
        zone_constraints = []

        and_constraints = []
        for j in range(len(points) - 1):
            and_constraints.append(is_left(x, y, points[j][0], points[j][2], points[j + 1][0], points[j + 1][2]))

        zone_constraints.append(And(and_constraints))

        #print(zone_constraints)
        ####### Minimum value of X range #######
        o = Optimize()
        o.add(zone_constraints)
        o.minimize(x)
        o.check()
        
        min_x_range = int(str(o.model()[x]))

        ####### Maximum value of X range #######
        o = Optimize()
        o.add(zone_constraints)
        o.maximize(x)
        o.check()
        #print(o.model()[x])

        max_x_range = int(str(o.model()[x]))
        
        
        for j in range(min_x_range, max_x_range):
            ####### Minimum value of Y range #######
            o = Optimize()
            o.add(zone_constraints)
            o.add(x == j)
            o.minimize(y)
            o.check()

            min_y_range = o.model()[y]
            if min_y_range == None:
                min_y_range = 0

            ####### Maximum value of Y range #######
            o = Optimize()
            o.add(zone_constraints)
            o.add(x == j)
            o.maximize(y)
            o.check()

            max_y_range = o.model()[y]
            if max_y_range == None:
                max_y_range = 0
                
            list_time_min[zone_id][j].append(int(str(min_y_range)))
            list_time_max[zone_id][j].append(int(str(max_y_range)))
    return list_time_min, list_time_max

# Range Calculation (House A)

In [24]:
num_zones = 5
list_time_min_house_A_occ_1, list_time_max_house_A_occ_1 = range_calculation(clusters_house_A_occ_1)
list_time_min_house_A_occ_2, list_time_max_house_A_occ_2 = range_calculation(clusters_house_A_occ_2)

# House-A Appliance Triggerring Cost Calculation

In [26]:
occ_1_zones = []
occ_2_zones = []

clean_dataframe_house_A = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-A.csv")

for i in range(len(clean_dataframe_house_A)):
    occ_1_zones.append(activity_zone_map[clean_dataframe_house_A['Occupant-1 Activity'][i]])
    occ_2_zones.append(activity_zone_map[clean_dataframe_house_A['Occupant-2 Activity'][i]])

clean_dataframe_house_A['Occupant-1 Zone'] = occ_1_zones
clean_dataframe_house_A['Occupant-2 Zone'] = occ_2_zones

In [27]:
clean_dataframe_house_A

,House,Day,Minute,Occupant-1 Activity,Occupant-2 Activity,Occupant-1 Zone,Occupant-2 Zone
0,A,1,0,12,17,2,1
1,A,1,1,12,17,2,1
2,A,1,2,12,17,2,1
3,A,1,3,12,17,2,1
4,A,1,4,12,17,2,1
...,...,...,...,...,...,...,...
43195,A,30,1435,12,2,2,0
43196,A,30,1436,12,2,2,0
43197,A,30,1437,12,2,2,0
43198,A,30,1438,12,2,2,0


In [37]:
day_wise_appl_cost_house_A_occ_1 = []
## house A occ 1
for i in range(0, 43200, 1440):
    cost = 0
    entrances = []
    prev_zone = clean_dataframe_house_A['Occupant-1 Zone'][i]
    attackable = np.zeros((1440))
    for j in range(i, i + 1440):
        zone_occupants = [0, 0, 0, 0, 0]
        zone_occupants[clean_dataframe_house_A['Occupant-1 Zone'][j]] += 1
        zone_occupants[clean_dataframe_house_A['Occupant-2 Zone'][j]] += 1
        # print(zone_occupants)
        if clean_dataframe_house_A['Occupant-1 Zone'][j] != prev_zone:
            entrances.append([j%1440, clean_dataframe_house_A['Occupant-1 Zone'][j]])
            prev_zone = clean_dataframe_house_A['Occupant-1 Zone'][j]
    for j in range(len(entrances) - 1):
        zone = entrances[j][1]
        time = entrances[j][0] 
        #print(entrances[j], zone, time)
        #print(list_time_min_house_A_occ_1[zone][time])
        if len(list_time_min_house_A_occ_1[zone][time]) == 0:
            #print("Here")
            attack_time = 0
        else:
            attack_time = min(entrances[j + 1][0] - entrances[j][0], list_time_min_house_A_occ_1[zone][time][0])
        #if entrances[j][1] == 2 or entrances[j][1] == 3 or entrances[j][1] == 4:
        if entrances[j][1] == 3 or entrances[j][1] == 4:
            attackable[entrances[j][0] : entrances[j][0] + attack_time] = 1
        #print(entrances[j][0], attack_time)
    for j in range(len(attackable)):
        if attackable[j] == 1:
            COST_PER_KWH = 0.1137
            #if house_A_occ_1_final_schedule[j][0] == 2 or house_A_occ_1_final_schedule[j][0] == 3 or house_A_occ_1_final_schedule[j][0] == 4:
            if house_A_occ_1_final_schedule[j][0] == 3 or house_A_occ_1_final_schedule[j][0] == 4:
                cost += (zone_cost[house_A_occ_1_final_schedule[j][0]] / 60000 ) * COST_PER_KWH
    day_wise_appl_cost_house_A_occ_1.append(cost)

In [38]:
day_wise_appl_cost_house_A_occ_2 = []
## house A occ 1
for i in range(0, 43200, 1440):
    cost = 0
    entrances = []
    prev_zone = clean_dataframe_house_A['Occupant-2 Zone'][i]
    attackable = np.zeros((1440))
    for j in range(i, i + 1440):
        zone_occupants = [0, 0, 0, 0, 0]
        zone_occupants[clean_dataframe_house_A['Occupant-1 Zone'][j]] += 1
        zone_occupants[clean_dataframe_house_A['Occupant-2 Zone'][j]] += 1
        # print(zone_occupants)
        if clean_dataframe_house_A['Occupant-2 Zone'][j] != prev_zone:
            entrances.append([j%1440, clean_dataframe_house_A['Occupant-2 Zone'][j]])
            prev_zone = clean_dataframe_house_A['Occupant-2 Zone'][j]
    for j in range(len(entrances) - 1):
        zone = entrances[j][1]
        time = entrances[j][0] 
        #print(entrances[j], zone, time)
        #print(list_time_min_house_A_occ_1[zone][time])
        if len(list_time_min_house_A_occ_2[zone][time]) == 0:
            #print("Here")
            attack_time = 0
        else:
            attack_time = min(entrances[j + 1][0] - entrances[j][0], list_time_min_house_A_occ_2[zone][time][0])
        #if entrances[j][1] == 2 or entrances[j][1] == 3 or entrances[j][1] == 4:
        if entrances[j][1] == 3 or entrances[j][1] == 4:
            attackable[entrances[j][0] : entrances[j][0] + attack_time] = 1
        #print(entrances[j][0], attack_time)
    for j in range(len(attackable)):
        if attackable[j] == 1:
            COST_PER_KWH = 0.1137 
            #if house_A_occ_1_final_schedule[j][0] == 2 or house_A_occ_1_final_schedule[j][0] == 3 or house_A_occ_1_final_schedule[j][0] == 4:
            if house_A_occ_1_final_schedule[j][0] == 3 or house_A_occ_1_final_schedule[j][0] == 4:
                cost += (zone_cost[house_A_occ_1_final_schedule[j][0]] / 60000 ) * COST_PER_KWH
    day_wise_appl_cost_house_A_occ_2.append(cost)

In [39]:
house_A_appl_cost = np.array(day_wise_appl_cost_house_A_occ_1) + np.array(day_wise_appl_cost_house_A_occ_2)

In [40]:
sum(house_A_appl_cost)

13.036864739999997